In [2]:
import sys  
sys.path.insert(1, './../../../ESN-IP/IP/.')

import random

random.seed(10)

from DATA import NARMA10

data = NARMA10(True, [80,0,20])

X_TR, Y_TR = data.TR()
X_VAL, Y_VAL = data.VAL()
X_TS, Y_TS = data.TS()


In [3]:
from Metrics import MSE
from IPReservoir import IPReservoir
from ESN import Reservoir, EchoStateNetwork
from IntrinsicPlasticity import IPMask, IPDistribution

N_UNITS_LIST = [100, 150, 200, 250]
W_range=(-0.5, 0.5)
initial_rho = 0.5
desired_rhos = [0.95, 0.95, 0.95, 0.95]
bias_range = [-0.1, 0.1]


In [4]:
REPETITIONS = 2
max_epochs = 15 
columns = ["Name", "TYPE", "MSE", "MC", "Lambda max", "delta_phI", "KL", "Rho", "A-Norm", "B-Norm"]
models_num = len(N_UNITS_LIST)*len(desired_rhos)
rows = []

for N_UNITS, desired_rho in zip(N_UNITS_LIST, desired_rhos):
    config_string = f"{N_UNITS} UNIT - Spectral Radius: {desired_rho}"
    names = ["Vanilla", "Gauss 1", "Gauss 2", "Bimodal", "Trim 1", "Trim 2"]
    identifiers = [ f"- {name} - {config_string} " for name in names]
    
    for R in range(REPETITIONS):
        vanilla_reservoir = Reservoir(N = N_UNITS, desired_rho = desired_rho, W_range=W_range, bias_range=bias_range)
        
        gaussian_reservoir_1 = IPReservoir(N = N_UNITS, desired_rho=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.gaussian(N_UNITS, 0.5, apply_activation=True))
        gaussian_reservoir_2 = IPReservoir(N = N_UNITS, desired_rho=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.gaussian(N_UNITS, 0.25, apply_activation=True))

        bimodal_reservoir = IPReservoir(N = N_UNITS, desired_rho=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.mixedBimodalMask(N_UNITS, 0.69, 0.92, apply_activation=True))
        
        trimodal_reservoir_1 = IPReservoir(N = N_UNITS, desired_rho=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.trimodal(N_UNITS, 9/10, 0.25, 0.15, 0.92, apply_activation=True))
        trimodal_reservoir_2 = IPReservoir(N = N_UNITS, desired_rho=initial_rho, bias = True, W_range=W_range, bias_range=bias_range, mask=IPMask.trimodal(N_UNITS, 9/10, 0.25, 0.075, 0.72, apply_activation=False))
        
        print("Traning IP models")
        gaussian_reservoir_1.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, eta=-0.0000025, verbose=False)
        gaussian_reservoir_2.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, eta=-0.0000025, verbose=False)
        bimodal_reservoir.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, eta=-0.0000025, verbose=False)
        trimodal_reservoir_1.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, eta=0.000005, verbose=False)
        trimodal_reservoir_2.safe_mode_pre_train(X_TR, max_epochs=max_epochs, max_rho=desired_rho, eta=0.000001, verbose=False)

        models = [vanilla_reservoir, gaussian_reservoir_1, gaussian_reservoir_2, bimodal_reservoir, trimodal_reservoir_1, trimodal_reservoir_2]

        for model_index in range(len(models)): 
            model = models[model_index]

            esn = EchoStateNetwork(model)
            esn.train(X_TR,Y_TR, 0)

            mse = esn.evaluate(X_TS, Y_TS, metric=MSE()).item()
            
            mc = esn.MemoryCapacity().item()
            
            lambda_max = model.LCE(X_TS)

            de_phi = model.de_fi()

            KL = model.kl_value.item() if model_index != 0 else None
            a_norm = model.a.norm().item() if model_index != 0 else 1
            b_norm = model.b.norm().item() if model_index != 0 else 0

            # columns = ["________________Name____________________","TYPE", "MSE","MC","Lambda max","delta_phI","KL", "Rho", "A-Norm", "B-Norm"]
            rows.append([f"{R+1}) {identifiers[model_index]}" , model_index, mse, mc, lambda_max, de_phi, KL, model.rho().item(), a_norm, b_norm])
            print(rows[-1])

        
        

Traning IP models
Reservoir warmed up with the first 100 time steps


C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.17422e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


['1) - Vanilla - 100 UNIT - Spectral Radius: 0.95 ', 0, 0.02153116552919781, 17.99212074279785, -0.112855725, 0.09531324354183994, None, 0.949995756149292, 1, 0]
Reservoir warmed up with the first 100 time steps
['1) - Gauss 1 - 100 UNIT - Spectral Radius: 0.95 ', 1, 0.021900779182507556, 10.66826057434082, -1.1541387, 0.04338171573592435, 0.08365268260240555, 0.3154613971710205, 6.280044078826904, 0.4222005605697632]
Reservoir warmed up with the first 100 time steps
['1) - Gauss 2 - 100 UNIT - Spectral Radius: 0.95 ', 2, 0.021438735482658067, 11.078691482543945, -1.1742675, 0.04336686974080098, 0.027617046609520912, 0.3092270791530609, 6.198600769042969, 0.9857455492019653]
Reservoir warmed up with the first 100 time steps
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\casa_\AppData\Local\Temp\ipykernel_12988\4001907511.py", line 40, in <module>
    mc = esn.MemoryCapacity().item()
  File "c:\Users\casa_\OneDrive\Desktop\workspace\ESN-IP\NOTEBOOKS\BENCHMARKS\./../../../ESN-IP/IP/.\ESN.py", line 281, in MemoryCapacity
  File "c:\Users\casa_\OneDrive\Desktop\workspace\ESN-IP\NOTEBOOKS\BENCHMARKS\./../../../ESN-IP/IP/.\ESN.py", line 237, in train
  File "c:\Users\casa_\OneDrive\Desktop\workspace\ESN-IP\NOTEBOOKS\BENCHMARKS\./../../../ESN-IP/IP/.\ESN.py", line 189, in train
    return (torch.sum(state_eigs)**2)/torch.sum(state_eigs**2)
  File "C:\Users\casa_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-pack

In [39]:
import pandas as pd 
df = pd.DataFrame(rows)
df.to_csv("./RESULTS_NARMA10.csv", header=columns)

In [42]:
df = pd.DataFrame(rows, columns=columns)

In [43]:
df.groupby("TYPE").mean()

,MSE,MC,Lambda max,delta_phI,KL,Rho,A-Norm,B-Norm
TYPE,,,,,,,,
0,0.021832,20.968672,-0.119658,0.063288,NaN,0.950000,1.000000,0.000000
1,0.022020,10.310652,-0.920700,0.241393,0.251774,0.943127,22.615183,6.309521
2,0.033346,9.549275,-1.379600,0.127523,0.227529,0.937140,23.068914,8.904370
3,0.021907,9.002080,-0.791319,0.188575,0.097414,0.947827,24.938432,12.554076
4,0.021839,10.401893,-0.273329,0.127452,0.083611,0.944889,24.570249,10.895304
5,0.021876,10.737753,-0.091002,0.087818,0.056141,0.945963,25.500667,10.932029
